In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

AttributeError: module 'tensorflow' has no attribute 'config'

In [1]:
import os
import time
import random
import numpy as np
import scipy, multiprocessing
import tensorflow as tf
import tensorlayer as tl
from model import get_G, get_D
from config import config


In [2]:
from tensorlayer.layers import (Input, Conv2d,  Elementwise, SubpixelConv2d, Flatten, Dense)
from tensorlayer.models import Model

ImportError: cannot import name 'Elementwise'

In [ ]:
###====================== HYPER-PARAMETERS ===========================###

batch_size = config.TRAIN.batch_size  # use 8 if your GPU memory is small, and change [4, 4] in tl.vis.save_images to [2, 4]
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init

n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
shuffle_buffer_size = 128


In [3]:
# create folders to save result images and trained models
save_dir = "samples"
tl.files.exists_or_mkdir(save_dir)
checkpoint_dir = "checkpoint"
tl.files.exists_or_mkdir(checkpoint_dir)

[TL] [!] samples exists ...
[TL] [!] checkpoint exists ...


True

In [4]:
def get_train_data():
    # load dataset
    train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))[0:200]

    ## If your machine have enough memory, please pre-load the entire train set.
    train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)
        
    # dataset API and augmentation
    def generator_train():
        for img in train_hr_imgs:
            yield img
    def _map_fn_train(img):
        hr_patch = tf.image.random_crop(img, [384, 384, 3])
        hr_patch = hr_patch / (255. / 2.)
        hr_patch = hr_patch - 1.
        hr_patch = tf.image.random_flip_left_right(hr_patch)
        lr_patch = downscale_hr_patches(hr_patch)
        return lr_patch, hr_patch
    train_ds = tf.data.Dataset.from_generator(generator_train, output_types=(tf.float32))
    train_ds = train_ds.map(_map_fn_train, num_parallel_calls=multiprocessing.cpu_count())
        # train_ds = train_ds.repeat(n_epoch_init + n_epoch)
    train_ds = train_ds.shuffle(shuffle_buffer_size)
    train_ds = train_ds.prefetch(buffer_size=2)
    train_ds = train_ds.batch(batch_size)
        # value = train_ds.make_one_shot_iterator().get_next()
    return train_ds

def downscale_hr_patches(hr_patch):
    return tf.image.resize(hr_patch, size=[96, 96])

In [5]:
G = get_G((batch_size, 96, 96, 3))
D = get_D((batch_size, 384, 384, 3))
VGG = tl.models.vgg19(pretrained=True, end_with='pool4', mode='static')

lr_v = tf.Variable(lr_init)
g_optimizer_init = tf.optimizers.Adam(lr_v, beta_1=beta1)
g_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)
d_optimizer = tf.optimizers.Adam(lr_v, beta_1=beta1)

G.train()
D.train()
VGG.train()

train_ds = get_train_data()

[TL] Input  _inputlayer_1: (8, 96, 96, 3)
[TL] Conv2d conv2d_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d conv2d_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_3: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_2: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_1: fn: add act: No Activation
[TL] Conv2d conv2d_4: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_3: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d conv2d_5: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_4: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] Elementwise elementwise_2

[TL] BatchNorm batchnorm2d_34: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_40: n_filter: 256 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_35: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_41: n_filter: 512 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_36: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_42: n_filter: 1024 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_37: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_43: n_filter: 2048 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNorm batchnorm2d_38: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: False
[TL] Conv2d conv2d_44: n_filter: 1024 filter_size: (1, 1) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNo

In [ ]:
#G

In [ ]:
#D

In [6]:
## initialize learning (G)
n_step_epoch = round(n_epoch_init // batch_size)
for epoch in range(n_epoch_init):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape() as tape:
            fake_hr_patchs = G(lr_patchs)

            mse_f_lr_p = 0.0
            if config.DOWNSCALE_COMPARE : 
                fake_lr_patches = downscale_hr_patches(fake_hr_patchs)

                mse_f_lr_p = tl.cost.mean_squared_error(fake_lr_patches, lr_patchs, is_mean=True)

            mse_loss = tl.cost.mean_squared_error(fake_hr_patchs, hr_patchs, is_mean=True) + mse_f_lr_p

        grad = tape.gradient(mse_loss, G.trainable_weights)
        g_optimizer_init.apply_gradients(zip(grad, G.trainable_weights))
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, mse: {:.3f} ".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss))
    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_hr_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_init_{}.png'.format(epoch)))
        

Epoch: [0/100] step: [0/12] time: 0.977s, mse: 0.819 
Epoch: [0/100] step: [1/12] time: 0.469s, mse: 0.435 
Epoch: [0/100] step: [2/12] time: 0.540s, mse: 0.510 
Epoch: [0/100] step: [3/12] time: 0.414s, mse: 0.593 
Epoch: [0/100] step: [4/12] time: 0.445s, mse: 0.647 
Epoch: [0/100] step: [5/12] time: 0.465s, mse: 0.534 
Epoch: [0/100] step: [6/12] time: 0.448s, mse: 0.602 
Epoch: [0/100] step: [7/12] time: 0.432s, mse: 0.577 
Epoch: [0/100] step: [8/12] time: 0.350s, mse: 0.536 
Epoch: [0/100] step: [9/12] time: 0.351s, mse: 0.864 
Epoch: [0/100] step: [10/12] time: 0.347s, mse: 0.712 
Epoch: [0/100] step: [11/12] time: 0.348s, mse: 0.615 
Epoch: [0/100] step: [12/12] time: 0.449s, mse: 0.396 
Epoch: [0/100] step: [13/12] time: 0.351s, mse: 0.483 
Epoch: [0/100] step: [14/12] time: 0.357s, mse: 0.520 
Epoch: [0/100] step: [15/12] time: 0.360s, mse: 0.553 
Epoch: [0/100] step: [16/12] time: 0.356s, mse: 0.466 
Epoch: [0/100] step: [17/12] time: 0.356s, mse: 0.289 
Epoch: [0/100] step:

Epoch: [6/100] step: [1/12] time: 0.439s, mse: 0.087 
Epoch: [6/100] step: [2/12] time: 0.440s, mse: 0.102 
Epoch: [6/100] step: [3/12] time: 0.404s, mse: 0.081 
Epoch: [6/100] step: [4/12] time: 0.603s, mse: 0.089 
Epoch: [6/100] step: [5/12] time: 0.421s, mse: 0.084 
Epoch: [6/100] step: [6/12] time: 0.443s, mse: 0.090 
Epoch: [6/100] step: [7/12] time: 0.422s, mse: 0.057 
Epoch: [6/100] step: [8/12] time: 0.357s, mse: 0.083 
Epoch: [6/100] step: [9/12] time: 0.354s, mse: 0.097 
Epoch: [6/100] step: [10/12] time: 0.353s, mse: 0.057 
Epoch: [6/100] step: [11/12] time: 0.355s, mse: 0.058 
Epoch: [6/100] step: [12/12] time: 0.351s, mse: 0.072 
Epoch: [6/100] step: [13/12] time: 0.354s, mse: 0.075 
Epoch: [6/100] step: [14/12] time: 0.538s, mse: 0.076 
Epoch: [6/100] step: [15/12] time: 0.356s, mse: 0.092 
Epoch: [6/100] step: [16/12] time: 0.350s, mse: 0.071 
Epoch: [6/100] step: [17/12] time: 0.352s, mse: 0.058 
Epoch: [6/100] step: [18/12] time: 0.350s, mse: 0.102 
Epoch: [6/100] step

Epoch: [12/100] step: [1/12] time: 0.389s, mse: 0.076 
Epoch: [12/100] step: [2/12] time: 0.428s, mse: 0.051 
Epoch: [12/100] step: [3/12] time: 0.406s, mse: 0.054 
Epoch: [12/100] step: [4/12] time: 0.563s, mse: 0.059 
Epoch: [12/100] step: [5/12] time: 0.447s, mse: 0.060 
Epoch: [12/100] step: [6/12] time: 0.439s, mse: 0.044 
Epoch: [12/100] step: [7/12] time: 0.452s, mse: 0.057 
Epoch: [12/100] step: [8/12] time: 0.354s, mse: 0.045 
Epoch: [12/100] step: [9/12] time: 0.355s, mse: 0.130 
Epoch: [12/100] step: [10/12] time: 0.351s, mse: 0.032 
Epoch: [12/100] step: [11/12] time: 0.347s, mse: 0.069 
Epoch: [12/100] step: [12/12] time: 0.367s, mse: 0.065 
Epoch: [12/100] step: [13/12] time: 0.355s, mse: 0.054 
Epoch: [12/100] step: [14/12] time: 0.510s, mse: 0.067 
Epoch: [12/100] step: [15/12] time: 0.348s, mse: 0.070 
Epoch: [12/100] step: [16/12] time: 0.350s, mse: 0.084 
Epoch: [12/100] step: [17/12] time: 0.354s, mse: 0.058 
Epoch: [12/100] step: [18/12] time: 0.358s, mse: 0.057 
E

Epoch: [18/100] step: [0/12] time: 0.403s, mse: 0.055 
Epoch: [18/100] step: [1/12] time: 0.431s, mse: 0.042 
Epoch: [18/100] step: [2/12] time: 0.451s, mse: 0.075 
Epoch: [18/100] step: [3/12] time: 0.432s, mse: 0.072 
Epoch: [18/100] step: [4/12] time: 0.588s, mse: 0.048 
Epoch: [18/100] step: [5/12] time: 0.451s, mse: 0.045 
Epoch: [18/100] step: [6/12] time: 0.468s, mse: 0.041 
Epoch: [18/100] step: [7/12] time: 0.443s, mse: 0.034 
Epoch: [18/100] step: [8/12] time: 0.359s, mse: 0.038 
Epoch: [18/100] step: [9/12] time: 0.358s, mse: 0.055 
Epoch: [18/100] step: [10/12] time: 0.359s, mse: 0.043 
Epoch: [18/100] step: [11/12] time: 0.376s, mse: 0.037 
Epoch: [18/100] step: [12/12] time: 0.347s, mse: 0.050 
Epoch: [18/100] step: [13/12] time: 0.352s, mse: 0.033 
Epoch: [18/100] step: [14/12] time: 0.503s, mse: 0.042 
Epoch: [18/100] step: [15/12] time: 0.368s, mse: 0.055 
Epoch: [18/100] step: [16/12] time: 0.356s, mse: 0.039 
Epoch: [18/100] step: [17/12] time: 0.353s, mse: 0.033 
Ep

Epoch: [23/100] step: [23/12] time: 0.355s, mse: 0.042 
Epoch: [23/100] step: [24/12] time: 0.352s, mse: 0.026 
Epoch: [24/100] step: [0/12] time: 0.461s, mse: 0.049 
Epoch: [24/100] step: [1/12] time: 0.462s, mse: 0.037 
Epoch: [24/100] step: [2/12] time: 0.451s, mse: 0.044 
Epoch: [24/100] step: [3/12] time: 0.418s, mse: 0.045 
Epoch: [24/100] step: [4/12] time: 0.576s, mse: 0.051 
Epoch: [24/100] step: [5/12] time: 0.454s, mse: 0.045 
Epoch: [24/100] step: [6/12] time: 0.452s, mse: 0.033 
Epoch: [24/100] step: [7/12] time: 0.425s, mse: 0.028 
Epoch: [24/100] step: [8/12] time: 0.346s, mse: 0.027 
Epoch: [24/100] step: [9/12] time: 0.345s, mse: 0.031 
Epoch: [24/100] step: [10/12] time: 0.368s, mse: 0.038 
Epoch: [24/100] step: [11/12] time: 0.354s, mse: 0.041 
Epoch: [24/100] step: [12/12] time: 0.351s, mse: 0.038 
Epoch: [24/100] step: [13/12] time: 0.358s, mse: 0.037 
Epoch: [24/100] step: [14/12] time: 0.503s, mse: 0.034 
Epoch: [24/100] step: [15/12] time: 0.350s, mse: 0.038 
Ep

...

Epoch: [71/100] step: [8/12] time: 0.348s, mse: 0.020 
Epoch: [71/100] step: [9/12] time: 0.522s, mse: 0.025 
Epoch: [71/100] step: [10/12] time: 0.350s, mse: 0.026 
Epoch: [71/100] step: [11/12] time: 0.353s, mse: 0.031 
Epoch: [71/100] step: [12/12] time: 0.359s, mse: 0.013 
Epoch: [71/100] step: [13/12] time: 0.354s, mse: 0.016 
Epoch: [71/100] step: [14/12] time: 0.354s, mse: 0.023 
Epoch: [71/100] step: [15/12] time: 0.354s, mse: 0.021 
Epoch: [71/100] step: [16/12] time: 0.349s, mse: 0.023 
Epoch: [71/100] step: [17/12] time: 0.349s, mse: 0.023 
Epoch: [71/100] step: [18/12] time: 0.352s, mse: 0.032 
Epoch: [71/100] step: [19/12] time: 0.460s, mse: 0.021 
Epoch: [71/100] step: [20/12] time: 0.345s, mse: 0.035 
Epoch: [71/100] step: [21/12] time: 0.353s, mse: 0.022 
Epoch: [71/100] step: [22/12] time: 0.350s, mse: 0.017 
Epoch: [71/100] step: [23/12] time: 0.362s, mse: 0.031 
Epoch: [71/100] step: [24/12] time: 0.360s, mse: 0.017 
Epoch: [72/100] step: [0/12] time: 0.403s, mse: 0.

Epoch: [77/100] step: [6/12] time: 0.425s, mse: 0.032 
Epoch: [77/100] step: [7/12] time: 0.441s, mse: 0.027 
Epoch: [77/100] step: [8/12] time: 0.350s, mse: 0.016 
Epoch: [77/100] step: [9/12] time: 0.537s, mse: 0.024 
Epoch: [77/100] step: [10/12] time: 0.359s, mse: 0.023 
Epoch: [77/100] step: [11/12] time: 0.359s, mse: 0.040 
Epoch: [77/100] step: [12/12] time: 0.380s, mse: 0.022 
Epoch: [77/100] step: [13/12] time: 0.359s, mse: 0.013 
Epoch: [77/100] step: [14/12] time: 0.350s, mse: 0.022 
Epoch: [77/100] step: [15/12] time: 0.353s, mse: 0.021 
Epoch: [77/100] step: [16/12] time: 0.350s, mse: 0.047 
Epoch: [77/100] step: [17/12] time: 0.357s, mse: 0.024 
Epoch: [77/100] step: [18/12] time: 0.357s, mse: 0.023 
Epoch: [77/100] step: [19/12] time: 0.487s, mse: 0.022 
Epoch: [77/100] step: [20/12] time: 0.354s, mse: 0.054 
Epoch: [77/100] step: [21/12] time: 0.356s, mse: 0.022 
Epoch: [77/100] step: [22/12] time: 0.352s, mse: 0.017 
Epoch: [77/100] step: [23/12] time: 0.352s, mse: 0.0

Epoch: [83/100] step: [4/12] time: 0.424s, mse: 0.023 
Epoch: [83/100] step: [5/12] time: 0.416s, mse: 0.040 
Epoch: [83/100] step: [6/12] time: 0.437s, mse: 0.034 
Epoch: [83/100] step: [7/12] time: 0.397s, mse: 0.028 
Epoch: [83/100] step: [8/12] time: 0.354s, mse: 0.018 
Epoch: [83/100] step: [9/12] time: 0.462s, mse: 0.021 
Epoch: [83/100] step: [10/12] time: 0.354s, mse: 0.017 
Epoch: [83/100] step: [11/12] time: 0.346s, mse: 0.029 
Epoch: [83/100] step: [12/12] time: 0.352s, mse: 0.021 
Epoch: [83/100] step: [13/12] time: 0.356s, mse: 0.016 
Epoch: [83/100] step: [14/12] time: 0.349s, mse: 0.033 
Epoch: [83/100] step: [15/12] time: 0.350s, mse: 0.025 
Epoch: [83/100] step: [16/12] time: 0.357s, mse: 0.032 
Epoch: [83/100] step: [17/12] time: 0.352s, mse: 0.019 
Epoch: [83/100] step: [18/12] time: 0.357s, mse: 0.024 
Epoch: [83/100] step: [19/12] time: 0.460s, mse: 0.028 
Epoch: [83/100] step: [20/12] time: 0.348s, mse: 0.050 
Epoch: [83/100] step: [21/12] time: 0.363s, mse: 0.019

Epoch: [89/100] step: [2/12] time: 0.427s, mse: 0.023 
Epoch: [89/100] step: [3/12] time: 0.419s, mse: 0.022 
Epoch: [89/100] step: [4/12] time: 0.444s, mse: 0.027 
Epoch: [89/100] step: [5/12] time: 0.428s, mse: 0.024 
Epoch: [89/100] step: [8/12] time: 0.360s, mse: 0.016 
Epoch: [89/100] step: [9/12] time: 0.532s, mse: 0.028 
Epoch: [89/100] step: [10/12] time: 0.360s, mse: 0.025 
Epoch: [89/100] step: [11/12] time: 0.351s, mse: 0.035 
Epoch: [89/100] step: [12/12] time: 0.350s, mse: 0.018 
Epoch: [89/100] step: [13/12] time: 0.355s, mse: 0.016 
Epoch: [89/100] step: [14/12] time: 0.361s, mse: 0.024 
Epoch: [89/100] step: [15/12] time: 0.348s, mse: 0.025 
Epoch: [89/100] step: [16/12] time: 0.356s, mse: 0.033 
Epoch: [89/100] step: [17/12] time: 0.351s, mse: 0.025 
Epoch: [89/100] step: [18/12] time: 0.352s, mse: 0.028 
Epoch: [89/100] step: [19/12] time: 0.521s, mse: 0.024 
Epoch: [89/100] step: [20/12] time: 0.358s, mse: 0.033 
Epoch: [89/100] step: [21/12] time: 0.363s, mse: 0.015

Epoch: [95/100] step: [2/12] time: 0.408s, mse: 0.017 
Epoch: [95/100] step: [3/12] time: 0.433s, mse: 0.021 
Epoch: [95/100] step: [4/12] time: 0.432s, mse: 0.018 
Epoch: [95/100] step: [5/12] time: 0.421s, mse: 0.029 
Epoch: [95/100] step: [6/12] time: 0.428s, mse: 0.020 
Epoch: [95/100] step: [7/12] time: 0.446s, mse: 0.014 
Epoch: [95/100] step: [8/12] time: 0.353s, mse: 0.016 
Epoch: [95/100] step: [9/12] time: 0.458s, mse: 0.032 
Epoch: [95/100] step: [10/12] time: 0.356s, mse: 0.022 
Epoch: [95/100] step: [11/12] time: 0.353s, mse: 0.032 
Epoch: [95/100] step: [12/12] time: 0.366s, mse: 0.022 
Epoch: [95/100] step: [13/12] time: 0.353s, mse: 0.018 
Epoch: [95/100] step: [14/12] time: 0.352s, mse: 0.019 
Epoch: [95/100] step: [15/12] time: 0.350s, mse: 0.021 
Epoch: [95/100] step: [16/12] time: 0.352s, mse: 0.018 
Epoch: [95/100] step: [17/12] time: 0.348s, mse: 0.019 
Epoch: [95/100] step: [18/12] time: 0.351s, mse: 0.030 
Epoch: [95/100] step: [19/12] time: 0.514s, mse: 0.015 


<tf.Tensor: id=11545250, shape=(8, 384, 384, 3), dtype=float32, numpy=
array([[[[-1.10720277e-01,  2.13976018e-05,  6.22878894e-02],
         [-9.11818370e-02,  2.41432386e-03,  6.69666901e-02],
         [-1.37739226e-01,  2.76623573e-02,  1.43918946e-01],
         ...,
         [-2.83510894e-01, -3.71649384e-01, -4.27615672e-01],
         [-2.49583766e-01, -2.83549696e-01, -3.24407995e-01],
         [-2.11769626e-01, -2.17196181e-01, -2.81192362e-01]],

        [[-1.02572523e-01,  7.70851038e-05,  1.01522297e-01],
         [-1.18275106e-01,  2.29819957e-02,  8.97525698e-02],
         [-1.29657865e-01,  6.68934062e-02,  2.08978757e-01],
         ...,
         [-3.70764226e-01, -4.40409750e-01, -5.14495134e-01],
         [-2.91253746e-01, -3.53471875e-01, -4.00750220e-01],
         [-2.16340318e-01, -2.77750701e-01, -3.63379806e-01]],

        [[-1.78886220e-01,  4.00671037e-03,  1.07955717e-01],
         [-1.75520137e-01,  1.26564484e-02,  1.35625690e-01],
         [-2.29224443e-01,  3

In [8]:
G.save_weights(os.path.join(checkpoint_dir, 'g-initial.h5'))

[TL] [*] Saving TL weights into checkpoint/g-initial.h5
[TL] [*] Saved


In [ ]:
## adversarial learning (G, D)
n_step_epoch = round(n_epoch // batch_size)
for epoch in range(n_epoch):
    for step, (lr_patchs, hr_patchs) in enumerate(train_ds):
        if lr_patchs.shape[0] != batch_size: # if the remaining data in this epoch < batch_size
            break
        step_time = time.time()
        with tf.GradientTape(persistent=True) as tape:
            fake_patchs = G(lr_patchs)
            logits_fake = D(fake_patchs)
            logits_real = D(hr_patchs)
            feature_fake = VGG((fake_patchs+1)/2.) # the pre-trained VGG uses the input range of [0, 1]
            feature_real = VGG((hr_patchs+1)/2.)
            d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real))
            d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake))
            d_loss = d_loss1 + d_loss2
            g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake))

            # mse_loss = tl.cost.mean_squared_error(fake_patchs, hr_patchs, is_mean=True)
            mse_f_lr_p = 0.0
            if config.DOWNSCALE_COMPARE: 
                fake_lr_patches = downscale_hr_patches(fake_hr_patchs)

                mse_f_lr_p = tl.cost.mean_squared_error(fake_lr_patches, lr_patchs, is_mean=True)

            mse_loss = tl.cost.mean_squared_error(fake_hr_patchs, hr_patchs, is_mean=True) + mse_f_lr_p

            vgg_loss = 2e-6 * tl.cost.mean_squared_error(feature_fake, feature_real, is_mean=True)
            g_loss = mse_loss + vgg_loss + g_gan_loss
        grad = tape.gradient(g_loss, G.trainable_weights)
        g_optimizer.apply_gradients(zip(grad, G.trainable_weights))
        grad = tape.gradient(d_loss, D.trainable_weights)
        d_optimizer.apply_gradients(zip(grad, D.trainable_weights))
        
        print("Epoch: [{}/{}] step: [{}/{}] time: {:.3f}s, g_loss(mse:{:.3f}, vgg:{:.3f}, adv:{:.3f}) d_loss: {:.3f}".format(
            epoch, n_epoch_init, step, n_step_epoch, time.time() - step_time, mse_loss, vgg_loss, g_gan_loss, d_loss))

    # update the learning rate
    if epoch != 0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay**(epoch // decay_every)
        lr_v.assign(lr_init * new_lr_decay)
        log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
        print(log)

    if (epoch != 0) and (epoch % 10 == 0):
        tl.vis.save_images(fake_patchs.numpy(), [2, 4], os.path.join(save_dir, 'train_g_{}.png'.format(epoch)))
        G.save_weights(os.path.join(checkpoint_dir, 'g-{epoch}.h5'.format(epoch=epoch)))
        D.save_weights(os.path.join(checkpoint_dir, 'd-{epoch}.h5'.format(epoch=epoch)))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch: [0/100] step: [0/125] time: 3.354s, g_loss(mse:1.105, vgg:0.058, adv:0.001) d_loss: 1.862
Epoch: [0/100] step: [1/125] time: 1.497s, g_loss(mse:0.880, vgg:0.055, adv:0.002) d_loss: 4.273
Epoch: [0/100] step: [2/125] time: 1.469s, g_loss(mse:0.903, vgg:0.049, adv:0.001) d_loss: 4.288
Epoch: [0/100] step: [3/125] time: 1.595s, g_loss(mse:0.877, vgg:0.055, adv:0.001) d_loss: 3.096
Epoch: [0/100] step: [4/125] time: 1.483s, g_loss(mse:0.975, vgg:0.054, adv:0.002) d_loss: 2.195
Epoch: [0/100] step: [5/125] time: 1.453s, g_loss(mse:1.310, vgg:0.083, adv:0.001) d_loss: 2.191
Epoch: [0/100] step: [6/125] time: 1.516s, g_loss(mse:1.032, vgg:0.065, adv:0.001) d_loss: 2.197
Epoch: [0/100] step: [7/125] time: 1.487s, g_loss(mse:1.328, vgg:0.040, adv:0.002) d_loss: 4.459
Epoch: [0/100] step: [8/125] time: 1.409s, g_loss(mse:1.203, vgg:0.035, adv:0.001) d_loss: 1.903
Epoch: [0/100] step: [9/125] time

Epoch: [3/100] step: [6/125] time: 1.483s, g_loss(mse:1.210, vgg:0.041, adv:0.001) d_loss: 1.939
Epoch: [3/100] step: [7/125] time: 1.429s, g_loss(mse:1.469, vgg:0.025, adv:0.002) d_loss: 2.717
Epoch: [3/100] step: [8/125] time: 1.411s, g_loss(mse:0.976, vgg:0.036, adv:0.002) d_loss: 3.329
Epoch: [3/100] step: [9/125] time: 1.416s, g_loss(mse:0.865, vgg:0.048, adv:0.001) d_loss: 3.052
Epoch: [3/100] step: [10/125] time: 1.425s, g_loss(mse:1.574, vgg:0.039, adv:0.001) d_loss: 3.130
Epoch: [3/100] step: [11/125] time: 1.554s, g_loss(mse:1.574, vgg:0.043, adv:0.001) d_loss: 2.599
Epoch: [3/100] step: [12/125] time: 1.406s, g_loss(mse:1.100, vgg:0.036, adv:0.002) d_loss: 3.154
Epoch: [3/100] step: [13/125] time: 1.418s, g_loss(mse:1.143, vgg:0.030, adv:0.002) d_loss: 2.431
Epoch: [3/100] step: [14/125] time: 1.417s, g_loss(mse:1.022, vgg:0.036, adv:0.002) d_loss: 2.200
Epoch: [3/100] step: [15/125] time: 1.388s, g_loss(mse:1.481, vgg:0.060, adv:0.002) d_loss: 2.512
Epoch: [3/100] step: [16

Epoch: [6/100] step: [15/125] time: 1.438s, g_loss(mse:1.339, vgg:0.059, adv:0.001) d_loss: 1.811
Epoch: [6/100] step: [16/125] time: 1.414s, g_loss(mse:1.089, vgg:0.056, adv:0.000) d_loss: 1.872
Epoch: [6/100] step: [17/125] time: 1.411s, g_loss(mse:1.298, vgg:0.032, adv:0.001) d_loss: 1.655
Epoch: [6/100] step: [18/125] time: 1.409s, g_loss(mse:1.308, vgg:0.063, adv:0.001) d_loss: 1.896
Epoch: [6/100] step: [19/125] time: 1.634s, g_loss(mse:1.701, vgg:0.056, adv:0.001) d_loss: 1.610
Epoch: [6/100] step: [20/125] time: 1.421s, g_loss(mse:0.901, vgg:0.043, adv:0.001) d_loss: 2.233
Epoch: [6/100] step: [21/125] time: 1.415s, g_loss(mse:1.394, vgg:0.032, adv:0.002) d_loss: 2.200
Epoch: [6/100] step: [22/125] time: 1.401s, g_loss(mse:1.153, vgg:0.043, adv:0.001) d_loss: 1.922
Epoch: [6/100] step: [23/125] time: 1.405s, g_loss(mse:0.918, vgg:0.058, adv:0.001) d_loss: 1.870
Epoch: [6/100] step: [24/125] time: 1.415s, g_loss(mse:0.607, vgg:0.043, adv:0.001) d_loss: 1.952
Epoch: [7/100] step: